In [ ]:
import numpy as np
import numpy.ma as ma
import re
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import time
from matplotlib.animation import ArtistAnimation, FFMpegWriter
import matplotlib.cm as cm
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import COASTLINE, LAND

bad_value = -999 
num_cols = 1440
num_rows = 720

In [ ]:
def get_longitudes(num):
    start = 0.125

    i=0
    longs = []
    while (i<num):
        longs.append(start)
        i+=1
        start+=0.25
    
    return longs

In [ ]:
def read_dataset(name):

    max_length = 1440

    f = open(name)
    lines = f.readlines()
    lines = lines[7:]

    splitted_dict = {}

    for i in lines:
        i = i.strip()
        line = re.split(r'[\t\n]',i)
        if (line[-1].find("** line too long **") != -1) or (line[-1] == "** line too long **"):
            line.pop() 
        
        temp = []
        for j in range(1, len(line)):
            temp.append(float(line[j]))
            
        splitted_dict[line[0]] = temp
    
    rain = []
    for i in splitted_dict.keys():

        length = len(splitted_dict[i])
        if (length < max_length):
            max_length = length
    
    for i in splitted_dict.keys():
        temp = []
        j = 0
        while (j<len(splitted_dict[i])):
            temp.append(splitted_dict[i][j])
            j+=1
        while (j<1440):
            temp.append(bad_value)
            j+=1
        rain.append(temp)

    rain = np.array(rain)
    return rain, max_length

In [ ]:
def load_data(strings):

    datasets = []
    min_length = 1440

    for i in strings:
        data, length = read_dataset(i)
        if (length < min_length):
            min_length = length
        
        datasets.append(data)
    
    return datasets,min_length


In [ ]:
def get_contour_lines(rain, ax, extent, levels):
    rain_masked = np.ma.masked_where(rain == bad_value, rain)
    lons = np.linspace(-180, 180, rain.shape[1])
    lats = np.linspace(-90, 90, rain.shape[0])
    cs = ax.contourf(lons, lats, rain_masked, levels=levels, transform=ccrs.PlateCarree(central_longitude=180), extend='both')
    ax.contour(lons, lats, rain_masked, levels=levels, colors='black', transform=ccrs.PlateCarree(central_longitude=180))
    ax.add_feature(COASTLINE, linewidth=0.5, edgecolor='black')
    ax.add_feature(LAND, facecolor='lightyellow')
    ax.gridlines(draw_labels=True)
    ax.gridlines(xlocs=range(-180, 181, 30), ylocs=range(-90, 91, 30), linestyle='--', color='k')
    ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER)
    ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
    ax.set_extent([-180, -150, -20, 20], ccrs.PlateCarree())
    return cs

def make_contour_line_plots(datasets, extent):
    for i, dataset in enumerate(datasets):
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=180))
        levels = [0, 0.5, 1, 2, 25]
        # levels = np.linspace(0, 25, 11)

        cs = get_contour_lines(dataset, ax, extent, levels)
        plt.colorbar(cs, ax=ax, orientation='vertical', shrink=0.5, pad=0.05)

        plt.title(f'Contour Plot {i+1}')
        plt.tight_layout()
        plt.show()

strings = ["./dataset/1_May_2015.txt","./dataset/11_May_2015.txt","./dataset/21_May_2015.txt","./dataset/1_Jun_2015.txt","./dataset/11_Jun_2015.txt",
           "./dataset/21_Jun_2015.txt","./dataset/1_Jul_2015.txt","./dataset/11_Jul_2015.txt","./dataset/21_Jul_2015.txt"]

datasets,min_length = load_data(strings)
extent = get_longitudes(num_cols)[min_length]
make_contour_line_plots(datasets, extent)
